In [1]:
from fitlins.interfaces.bids import ModelSpecLoader, LoadBIDSModel, BIDSSelect, BIDSDataSink

from fitlins.interfaces.nistats import DesignMatrix, FirstLevelModel, SecondLevelModel

from fitlins.interfaces.visualizations import (
        DesignPlot, DesignCorrelationPlot, ContrastMatrixPlot, GlassBrainPlot)
from fitlins.interfaces.utils import MergeAll, CollateWithMetadata

from bids.layout import BIDSLayout
from fitlins.utils.bids import load_all_specs

from bids.tests import get_test_data_path
from os.path import join
import json

/Users/jeanettemumford/miniconda3/envs/fitlins/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
from bids.modeling import BIDSStatsModelsGraph
database_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data/ds003/dbcache'
# This will need adjustment if dbcache doesn't already exist
layout = BIDSLayout(
    database_path=database_path
)

model_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data/model-ds0003_smdl.json'
graph = BIDSStatsModelsGraph(layout, model_path)    

kwargs = {'desc': 'preproc', 'space': 'MNI152NLin2009cAsym', 'subject': ['01', '02', '03']} 
graph.load_collections(**kwargs)


/Users/jeanettemumford/miniconda3/envs/fitlins/lib/python3.9/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [3]:
from bids.modeling import transformations as tm

# remove the convolve transform and all that are past it
# This only happens to run or subject nodes (Look up all node name possibilities)
sub_node = graph.get_node('subject')
sub_trans = sub_node.transformations #has transformer and instructions
sub_pybids_transforms = sub_trans['transformer'] #just the version of pybids transforms
apply_trans = tm.TransformerManager(sub_pybids_transforms)
collection_apply_to = sub_node._collections[0]

In [4]:
# To do:  make this a function
# Adjust the instructions
sub_trans_instructions = sub_trans['instructions']

transformation_names = [l['name'] for l in sub_trans_instructions]
if 'Convolve' in transformation_names:
    convolve_idx = transformation_names.index('Convolve')
    trimmed_instructions = sub_trans_instructions[:convolve_idx]
    convolve_inputs = sub_trans_instructions[convolve_idx]['input']
else:
    trimmed_instructions = sub_trans_instructions
    convolve_inputs = None

In [ ]:
trans_out = apply_trans.transform(collection_apply_to.clone(), trimmed_instructions)

#This should work, since I don't want the sparse stuff

#next step:
#I think I can use sub_node.model to create the "confound" file and "ev" files
# will also need to use sub_node.contrasts
# find all variables involved in contrasts first
# EV files will be the union of the contrast names and convolve_inputs
# I will need to track which EVs require convolution and which do not (for fsf)
# Maybe just the file name would be name_3column.txt and name_1column.txt
# Then set up contrasts

In [ ]:
# For later (figuring out how database_path is dealth with)
# From run.py for fitlins (so opts is the parsed args)
    if opts.database_path is None:
        database_path = Path(work_dir) / 'dbcache'
        reset_database = True
    else:
        database_path = opts.database_path
        reset_database = False
#Generally I will need to copy code from run.py for  lots of things here's derivatives

    derivatives = True if not opts.derivatives else opts.derivatives
    # Need this when specifying args directly (i.e. neuroscout)
    # god bless neuroscout, but let's make it work for others!
    if isinstance(derivatives, list) and len(derivatives) == 1:
        # WRONG AND EVIL to those who have spaces in their paths... bad bad practice
        # TODO - fix neuroscout
        derivatives = derivatives[0].split(" ")


In [ ]:
def check_db_path_make_layout(database_path):
    try:
        layout = BIDSLayout(database_path = database_path)
    except:
        layout = BIDSLayout(
            root = root_dir,
            database_path = database_path,
#Maybe use this (from fitlins)
    if opts.database_path is None:
        database_path = Path(work_dir) / 'dbcache'
        reset_database = True
    else:
        database_path = opts.database_path
        reset_database = False
    #I think the first entry is actually root = opts.bids_dir
    layout = bids.BIDSLayout(
        opts.bids_dir,
        derivatives=derivatives,
        database_path=database_path,
        reset_database=reset_database
    )


        )
            
            
        
    


In [ ]:

database_path_copy = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data_copy/ds003/dbcache'

layout = BIDSLayout(root='/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data_copy/ds003',
    derivatives='/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data_copy/ds003_fmriprep',
    database_path=database_path_copy,
    reset_database=True,
)

In [ ]:
# Start turning the above into a function or two
database_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data_copy/ds003/dbcache'
# root is only needed if dbcache hasn't been made already
root='/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data_copy/ds003'
desc = 'preproc'
space = 'MNI152NLin2009cAsym'
subject = ['01', '02', '03']

def func1(database_path, desc, space, subject):
    layout = BIDSLayout(
        database_path=database_path
    )

model_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data/model-ds0003_smdl.json'
graph = BIDSStatsModelsGraph(layout, model_path)

kwargs = {'desc': 'preproc', 'space': 'MNI152NLin2009cAsym', 'subject': ['01', '02', '03']} 
graph.load_collections(**kwargs)



In [ ]:
#Try to grab data path
#'desc': 'preproc', 'space': 'MNI152NLin2009cAsym',

junk1 = BIDSSelect(
            database_path=database_path,
            entities = {'subject': '01'},
            selectors={
                'suffix': 'bold',
                'desc': 'preproc',
                'space': 'MNI152NLin2009cAsym',
                'extension': ['.nii', '.nii.gz', '.dtseries.nii', '.func.gii'],
            },
        )
junk1.run()
print(junk1._results.get('bold_files'))




In [ ]:
# I should check how they deal with duplicates. or is it impossible?

layout = BIDSLayout.load(database_path=database_path)
filters = {
    'suffix': ['T1w', 'bold'],
    'subject': '01',
    'desc': 'preproc',
    'space': 'MNI152NLin2009cAsym',
    'extension': ['.nii', '.nii.gz', '.dtseries.nii', '.func.gii']
}
output = layout.get(
    **filters
)


In [ ]:
rt = trans_out.entities['RepetitionTime']
# User will specify output_dir
output_dir = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/Data/ds003/bids_sm_to_fsf'
feat_out_root=f'{output_dir}/feat_analyses'

#Debating putting all additions in a dictionary

fsf_settings = {}
fsf_settings['set fmri(level)'] = 1


In [ ]:
#Copy stub and open up for additions!



I'll need this later.  The function to replace reg directories.

In [ ]:
def replace_reg_dir(featdir, t1_standard):
    import os
    import numpy as np
    import shutil
    if not os.path.isdir(featdir):
        print('Input featdir directory does not exist')
        return
    if not os.path.exists(f'{featdir}/example_func.nii.gz'):
        print('example_func.nii.gz not found \n')
        print('Check that input is feat directory')
        return
    shutil.rmtree(f'{featdir}/reg')
    os.makedirs(f'{featdir}/reg')
    ident_mat = np.identity(4)
    mat_names = [
        'example_func2highres', 'example_func2standard', 'highres2example_func',
        'highres2standard', 'standard2example_func', 'standard2highres'
    ]
    for mat_name in mat_names:
        np.savetxt(f'{featdir}/reg/{mat_name}.mat', ident_mat, delimiter=" ")
    shutil.copy(
        f'{featdir}/example_func.nii.gz', f'{featdir}/reg/standard.nii.gz'
    )
    shutil.copy(
        t1_standard, f'{featdir}/reg/highres.nii.gz'
    )


featdir = '/Users/jeanettemumford/Dropbox/Research/Talks/MumfordBrainStats/ds008_R1.1.0_raw/sub002/model/test_withstats3.feat'
# Technically this is usually a T1 in subject space, but the standard space is what I think we'll want
# This is the bg_image in higher levels (averaged)
t1_standard = '/Users/jeanettemumford/Dropbox/Research/Talks/MumfordBrainStats/ds008_R1.1.0_raw/sub002/model/task001_run002.feat/reg/highres.nii.gz'
replace_reg_dir(featdir, t1_standard)

In [ ]:
#I might need this type of code to apply the filters
# From statsmodels.py node.run() code line 437,  Loop up inputs and collections
        # Filter inputs and collections if needed
collection = node._collections[0]

        if filters:
            inputs = [i for i in inputs if matches_entities(i, filters)]
            collections = [c for c in collections if matches_entities(c, filters)]

In [ ]:
def load_graph(graph, node=None, inputs=None, **filters):
    if node is None:
        node = graph.root_node

    specs = node.run(inputs, group_by=node.group_by, **filters)
    outputs = list(chain(*[s.contrasts for s in specs]))

    base_entities = graph.model["input"]

    all_specs = {
        node.name: [
            {
                'contrasts': [c._asdict() for c in spec.contrasts],
                'entities': {**base_entities, **spec.entities},
                'level': spec.node.level,
                'X': spec.X,
                'name': spec.node.name,
                'model': spec.node.model,
                # Metadata is only used in higher level models; save space
                'metadata': spec.metadata if spec.node.level != "run" else None,
            }
            for spec in specs
        ]
    }

    for child in node.children:
        all_specs.update(
            load_graph(graph, child.destination, outputs, **child.filter)
        )

    return all_specs

In [ ]:
#trouble shooting load_graph problem
root = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/ModelZoo/my_models/ds000005/'
database_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/ModelZoo/my_models/ds000005/dbcache'
reset_database = True
model_path = '/Users/jeanettemumford/Dropbox/Research/Projects/Fsf_converter/ModelZoo/my_models/test.json'


In [ ]:
layout = BIDSLayout(
        root = root,
        database_path=database_path,
        reset_database=reset_database
)

graph = BIDSStatsModelsGraph(layout, model_path)
graph.load_collections(scan_length=480)    



In [ ]:
#all_specs = load_graph(graph, node=None, inputs=None)

node = graph.root_node
inputs = None
specs = node.run(inputs, group_by=node.group_by)
outputs = list(chain(*[s.contrasts for s in specs]))

base_entities = graph.model["input"]

all_specs = {
    node.name: [
        {
            'contrasts': [c._asdict() for c in spec.contrasts],
            'entities': {**base_entities, **spec.entities},
            'level': spec.node.level,
            'X': spec.X,
            'name': spec.node.name,
            'model': spec.node.model,
            # Metadata is only used in higher level models; save space
            'metadata': spec.metadata if spec.node.level != "run" else None,
        }
        for spec in specs
    ]
}

#Now for run level (see if it works)
#for child in node.children:
#    all_specs.update(
#        load_graph(graph, child.destination, outputs, **child.filter)
#    )

node2 = node.children[0].destination
inputs2 = outputs
filter = node.children[0].filter

#here's the error...
specs2 = node2.run(inputs2, group_by=node2.group_by)
#outputs = list(chain(*[s.contrasts for s in specs]))

In [ ]:
def test(a=10, b=20, **kwargs):
    print(a*b)
    print(kwargs)

In [ ]:
# testing a nilearn thing quickly

from nilearn.glm.first_level.hemodynamic_models import _resample_regressor